In [12]:
import numpy as np 
import pandas as pd
import pymap3d as pm
import pickle
from controller import DisturbanceRejectionController

In [13]:
# Preprocess an entire file to be fed into our model
def process_file(filename):
    data_load_try = []

    with open(filename, "rb") as file:
        try:
            while True:
                data_load_try.append(pickle.load(file))
        except:
            pass 

    df = pd.DataFrame(data_load_try)
    next_wps = []
    for wp in df['remaining_waypoints']:
        next_wps.append(wp[0])

    df['next_wps'] = next_wps
    df = df.drop('remaining_waypoints', axis=1)
    xy_np = pm.ecef2enu(df['ecefx'].astype(float), df['ecefy'].astype(float), df['ecefz'].astype(float), df['lat'].astype(float)[0], df['lon'].astype(float)[0], 0)
    xy_np = np.array(xy_np).T
    df['x'] = xy_np[:,0]
    df['y'] = xy_np[:,1]
    df['next_wps'] = df['next_wps'].apply(tuple)
    df = df.drop_duplicates()
    df = df.reset_index(drop=True)
    
    wps = np.array(df['next_wps'].tolist())
    xwp, ywp, zwp = pm.geodetic2enu(wps[:,0], wps[:,1], wps[:,2], df['lat'].astype(float)[0], df['lon'].astype(float)[0], 0)
    df['xwp'] = xwp 
    df['ywp'] = ywp 
    df['zwp'] = wps[:,2] + 125 # seems off by a bit, temporary correction

    vxy_np = pm.ecef2enuv(df['vEcefx'].astype(float), df['vEcefy'].astype(float), df['vEcefz'].astype(float), df['lat'].astype(float)[0], df['lon'].astype(float)[0], deg=True)
    vxy_np = np.array(vxy_np).T
    df['vx'] = vxy_np[:,0]
    df['vy'] = vxy_np[:,1]
    df['vz'] = vxy_np[:,2]

    return df

In [14]:
filepath = '../../wind-estimation-control/src/data/luis_wind/e_shape_no_wind.pkl'

data = process_file(filepath)

In [11]:

pickle.load(filepath)

NameError: name 'pickle' is not defined